In [9]:
import time
tic = time.time()

In [10]:
import warnings
warnings.filterwarnings('ignore')

In [11]:
import pandas as pd
import numpy as np
from scipy import stats
import random
from sklearn.preprocessing import LabelEncoder
import os

In [12]:
def seed_everything(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
seed_everything(42)

In [13]:
if not os.path.exists('./feat_v6'):
    os.mkdir('./feat_v6')

In [ ]:
train_bankstat = pd.read_csv('../problems/problem_1/train/train_bankStatement.csv')
train_hehavior = pd.read_csv('../problems/problem_1/train/train_behaviors.csv')
train_creditbill = pd.read_csv('../problems/problem_1/train/train_creditBill.csv')
train_label = pd.read_csv('../problems/problem_1/train/train_label.csv')
train_profile = pd.read_csv('../problems/problem_1/train/train_profile.csv')

In [ ]:
# test_bankstat = pd.read_csv('../problems/problem_1/A/test_bankStatement_A.csv')
# test_hehavior = pd.read_csv('../problems/problem_1/A/test_behaviors_A.csv')
# test_creditbill = pd.read_csv('../problems/problem_1/A/test_creditBill_A.csv')
# test_profile = pd.read_csv('../problems/problem_1/A/test_profile_A.csv')
test_bankstat = pd.read_csv('../problems/problem_1/B/test_bankStatement_B.csv')
test_hehavior = pd.read_csv('../problems/problem_1/B/test_behaviors_B.csv')
test_creditbill = pd.read_csv('../problems/problem_1/B/test_creditBill_B.csv')
test_profile = pd.read_csv('../problems/problem_1/B/test_profile_B.csv')
test_profile.shape

In [ ]:
bankstat = pd.concat([train_bankstat, test_bankstat])
behavior = pd.concat([train_hehavior, test_hehavior])
creditbill = pd.concat([train_creditbill, test_creditbill])
profile = pd.concat([train_profile, test_profile])

In [ ]:
bankstat_cnt = bankstat.groupby('用户标识').size().reset_index().rename({0:'流水次数'}, axis=1)
bankstat_cnt1 = bankstat.groupby(['用户标识', '交易类型']).size().reset_index()
bankstat_cnt1 = bankstat_cnt1.pivot('用户标识', '交易类型', 0).reset_index().rename({0:'交易0次数', 1:'交易1次数'}, axis=1)
bankstat_cnt = bankstat_cnt.merge(bankstat_cnt1, 'left', '用户标识')
# for i in [0, 1]:
#     bankstat_cnt['交易%d次数占比' % i] = bankstat_cnt['交易%d次数' % i] / bankstat_cnt['流水次数']
bankstat_cnt1 = bankstat.groupby(['用户标识', '工资收入标记']).size().reset_index()
bankstat_cnt1 = bankstat_cnt1.pivot('用户标识', '工资收入标记', 0).reset_index().rename({0:'非工资次数', 1:'工资次数'}, axis=1)
bankstat_cnt = bankstat_cnt.merge(bankstat_cnt1, 'left', '用户标识')
for i in ['非', '']:
    bankstat_cnt['%s工资次数占比' % i] = bankstat_cnt['%s工资次数' % i] / bankstat_cnt['流水次数']

display(bankstat_cnt.head(3))
bankstat_cnt.to_hdf('./feats_v6/bankstat_cnt.h5', key='data', mode='w')

In [ ]:
# bankstat['流水时间d'] = bankstat['流水时间'] // (3600)
bankstat.sort_values(by='流水时间', inplace=True)

bankstat_time = bankstat.groupby(['用户标识'])['流水时间'].agg({'流水时间_min': 'min', '流水时间_max': 'max'}).reset_index()#})
bankstat_time['流水时间_gap'] = bankstat_time['流水时间_max'] - bankstat_time['流水时间_min']
# bankstat_time['流水时间_mm'] = (bankstat_time['流水时间_max'] + bankstat_time['流水时间_min']) / 2

display(bankstat_time.head(3))
bankstat_time.to_hdf('./feats_v6/bankstat_time.h5', key='data', mode='w')

In [ ]:
bankstat_time2 = bankstat.groupby(['用户标识', '工资收入标记'])['流水时间'].agg({'流水时间_min': 'min', '流水时间_max': 'max'}).reset_index()
bankstat_time2 = bankstat_time2.pivot('用户标识', '工资收入标记', ['流水时间_min', '流水时间_max']).reset_index()
bankstat_time2.columns = ['用户标识', '非工资_流水时间_min', '工资_流水时间_min', '非工资_流水时间_max', '工资_流水时间_max']
bankstat_time2['非工资_流水时间_gap'] = bankstat_time2['非工资_流水时间_max'] - bankstat_time2['非工资_流水时间_min']
bankstat_time2['工资_流水时间_gap'] = bankstat_time2['工资_流水时间_max'] - bankstat_time2['工资_流水时间_min']

display(bankstat_time2.head(3))
bankstat_time2.to_hdf('./feats_v6/bankstat_time2.h5', key='data', mode='w')

In [ ]:
bankstat_amt2 = bankstat.groupby(['用户标识', '交易类型'])['交易金额'].agg(['mean', 'std', 'min', 'sum', 'skew']).reset_index()
bankstat_amt2 = bankstat_amt2.pivot_table(index='用户标识', columns='交易类型', values=['mean', 'std', 'min', 'sum', 'skew']).reset_index().fillna(0)
bankstat_amt2.columns = ['用户标识'] + [('交易' + ty + '金额_' + st) for st in ['mean', 'min', 'std', 'sum', 'skew'] for ty in ['0', '1'] ]
bankstat_amt2['交易0金额占比'] = bankstat_amt2['交易0金额_sum'] / (bankstat_amt2['交易0金额_sum'] + bankstat_amt2['交易1金额_sum'])
bankstat_amt2['交易1金额占比'] = bankstat_amt2['交易1金额_sum'] / (bankstat_amt2['交易0金额_sum'] + bankstat_amt2['交易1金额_sum'])
# bankstat_amt2['交易差额'] = bankstat_amt2['交易0金额_sum'] - bankstat_amt2['交易1金额_sum']

display(bankstat_amt2.head(3))
bankstat_amt2.to_hdf('./feats_v6/bankstat_amt2.h5', key='data', mode='w')

In [ ]:
date_set = behavior['日期'].unique()
date_map = {}
date_int = []
for d in date_set:
    d_int = [int(y) for y in d.split('-')]
    date_map[d] = d_int[0] * 100 + d_int[1]
    date_int.append(d_int[0] * 100 + d_int[1])
date_int = sorted(date_int)
date_int_map = dict((d, i) for i, d in enumerate(date_int))

behavior['日期'] = behavior['日期'].map(date_map)
behavior['月'] = behavior['日期'] // 100
behavior['日期编码'] = behavior['日期'].map(date_int_map)
behavior.head()

In [ ]:
behavior_cnt = behavior.groupby(['用户标识', '月']).size().reset_index()
behavior_cnt = behavior_cnt.pivot('用户标识', '月', 0).reset_index() 
behavior_cnt.columns = ['用户标识'] + ['月%d次数' % i for i in range(12)]
behavior_cnt['行为总次数'] = behavior_cnt[['月%d次数' % i for i in range(12)]].sum(axis=1)
for i in range(12):
    behavior_cnt['月%d次数占比' % i] = behavior_cnt['月%d次数' % i] / behavior_cnt['行为总次数']
behavior_cnt['活跃月数'] = 12 - behavior_cnt[['月%d次数' % i for i in range(12)]].isna().sum(axis=1)
# behavior_cnt['月次数_skew'] = behavior_cnt[['月%d次数' % i for i in range(12)]].skew(axis=1)

del behavior_cnt['行为总次数']
display(behavior_cnt.head(3))
behavior_cnt.to_hdf('./feats_v6/behavior_cnt.h5', key='data', mode='w')

In [ ]:
behavior_cnt2 = behavior.groupby(['用户标识', '星期几']).size().reset_index()
behavior_cnt2 = behavior_cnt2.pivot('用户标识', '星期几', 0).reset_index() 
behavior_cnt2.columns = ['用户标识'] + ['星期%d次数' % i for i in range(7)]
behavior_cnt2['行为总次数'] = behavior_cnt2[['星期%d次数' % i for i in range(7)]].sum(axis=1)
for i in range(7):
    behavior_cnt2['星期%d次数占比' % i] = behavior_cnt2['星期%d次数' % i] / behavior_cnt2['行为总次数']
behavior_cnt2['活跃星期数'] = 7 - behavior_cnt2[['星期%d次数' % i for i in range(7)]].isna().sum(axis=1)
# behavior_cnt2['最活跃星期数'] = np.argmax(behavior_cnt2[['星期%d次数' % i for i in range(7)]].fillna(-1).values, axis=1)
# behavior_cnt2['星期次数_skew'] = behavior_cnt2[['星期%d次数' % i for i in range(7)]].skew(axis=1)
# behavior_cnt2['活跃天数'] = behavior.groupby('用户标识')['日期编码'].nunique()


# del behavior_cnt2['行为总次数']
display(behavior_cnt2.head(3))
behavior_cnt2.to_hdf('./feats_v6/behavior_cnt2.h5', key='data', mode='w')

In [ ]:
behavior_cnt3 = behavior.groupby(['用户标识', '行为类型']).size().reset_index()
behavior_cnt3 = behavior_cnt3.pivot('用户标识', '行为类型', 0).reset_index()
behavior_cnt3.columns = ['用户标识'] + ['行为类型%d次数' % i for i in range(8)]
behavior_cnt3['行为总次数'] = behavior_cnt3[['行为类型%d次数' % i for i in range(8)]].sum(axis=1)
for i in range(8):
    behavior_cnt3['行为类型%d次数占比' % i] = behavior_cnt3['行为类型%d次数' % i] / behavior_cnt3['行为总次数']

del behavior_cnt3['行为总次数']
display(behavior_cnt3.head(3))
behavior_cnt3.to_hdf('./feats_v6/behavior_cnt3.h5', key='data', mode='w')

In [ ]:
behavior_cnt4 = behavior.groupby(['用户标识', '子类型1']).size().reset_index()
behavior_cnt4 = behavior_cnt4.pivot('用户标识', '子类型1', 0).reset_index()
behavior_cnt4.columns = ['用户标识'] + ['子类型1_%d次数' % i for i in range(111)]
behavior_cnt4['行为总次数'] = behavior_cnt4[['子类型1_%d次数' % i for i in range(111)]].sum(axis=1)
tmp = behavior_cnt4.isnull().sum() / len(behavior_cnt4)
# behavior_cnt4['其他子类型1次数'] = behavior_cnt4[tmp[tmp > 0.9].index.tolist()].sum(axis=1)
behavior_cnt4.drop(columns=tmp[tmp > 0.9].index.tolist(), inplace=True)
# for i in tmp[tmp <= 0.9].index.tolist():
#     if i == '用户标识':
#         continue
#     behavior_cnt4['%s占比' % i] = behavior_cnt4['%s' % i] / behavior_cnt4['行为总次数']

del behavior_cnt4['行为总次数']
display(behavior_cnt4.head(3))
behavior_cnt4.to_hdf('./feats_v6/behavior_cnt4.h5', key='data', mode='w')

In [ ]:
behavior_time = behavior.groupby(['用户标识'])['日期编码'].agg(dict(('行为日期_' + st, st) for st in ['mean', 'std', 'max', 'min'])).reset_index()
behavior_time['行为日期_mm'] = (behavior_time['行为日期_max'] + behavior_time['行为日期_min']) / 2

display(behavior_time.head(3))
behavior_time.to_hdf('./feats_v6/behavior_time.h5', key='data', mode='w')

In [ ]:
credit_time_min = creditbill.groupby(['用户标识','银行标识'])['账单时间戳'].min().reset_index()
credit_time_min = credit_time_min.pivot(index='用户标识', columns='银行标识', values='账单时间戳').reset_index()
credit_time_min.columns = ['用户标识'] + ['银行标识{}_账单时间戳_min'.format(i) for i in range(13)]
display(credit_time_min.head(3))

credit_time_min.to_hdf('./feats_v6/credit_time_min.h5', key='data', mode='w')

In [ ]:
credit_time_max = creditbill.groupby(['用户标识','银行标识'])['账单时间戳'].max().reset_index()
credit_time_max = credit_time_max.pivot(index='用户标识', columns='银行标识', values='账单时间戳').reset_index()
credit_time_max.columns = ['用户标识'] + ['银行标识{}_账单时间戳_max'.format(i) for i in range(13)]
display(credit_time_max.head(3))
# 
credit_time_max.to_hdf('./feats_v6/credit_time_max.h5', key='data', mode='w')

In [ ]:
credit_time_std = creditbill[creditbill['账单时间戳'] > 0].groupby(['用户标识','银行标识'])['账单时间戳'].std().reset_index()
credit_time_std = credit_time_std.pivot(index='用户标识', columns='银行标识', values='账单时间戳').reset_index()
credit_time_std.columns = ['用户标识'] + ['银行标识{}_账单时间戳_std'.format(i) for i in range(13)]
display(credit_time_std.head(3))

credit_time_std.to_hdf('./feats_v6/credit_time_std.h5', key='data', mode='w')

In [ ]:
credit_time_skew = creditbill.groupby(['用户标识','银行标识'])['账单时间戳'].skew().reset_index()
credit_time_skew = credit_time_skew.pivot(index='用户标识', columns='银行标识', values='账单时间戳').reset_index()
credit_time_skew.columns = ['用户标识'] + ['银行标识{}_账单时间戳_skew'.format(i) for i in range(13)]
display(credit_time_skew.head(3))

credit_time_skew.to_hdf('./feats_v6/credit_time_skew.h5', key='data', mode='w')

In [ ]:
credit_time_min = pd.read_hdf('./feats_v6/credit_time_min.h5', key='data')  
credit_time_max = pd.read_hdf('./feats_v6/credit_time_max.h5', key='data') 
gap_arr = credit_time_max.values[:, 1:] - credit_time_min.values[:, 1:]
credit_time_gap = pd.DataFrame(gap_arr, columns=['银行标识{}_账单时间戳_gap'.format(i) for i in range(13)])
credit_time_gap['用户标识'] = credit_time_min['用户标识']
display(credit_time_gap.head(3))

credit_time_gap.to_hdf('./feats_v6/credit_time_gap.h5', key='data', mode='w')

In [ ]:
credit_time_min = pd.read_hdf('./feats_v6/credit_time_min.h5', key='data')  
credit_time_max = pd.read_hdf('./feats_v6/credit_time_max.h5', key='data') 
mm2_arr = (credit_time_max.values[:, 1:] + credit_time_min.values[:, 1:]) / 2
credit_time_mm2 = pd.DataFrame(mm2_arr, columns=['银行标识{}_账单时间戳_mm2'.format(i) for i in range(13)])
credit_time_mm2['用户标识'] = credit_time_min['用户标识']
display(credit_time_mm2.head(3))

credit_time_mm2.to_hdf('./feats_v6/credit_time_mm2.h5', key='data', mode='w')

In [ ]:
creditbill.sort_values(by='账单时间戳',inplace=True)
creditbill['上期账单金额-上期还款金额'] = creditbill['上期账单金额'] - creditbill['上期还款金额']
creditbill['上期账单金额-本期账单余额'] = creditbill['上期账单金额'] - creditbill['本期账单余额']
creditbill['上期账单金额-信用卡额度'] = creditbill['上期账单金额'] - creditbill['信用卡额度']
creditbill['上期还款金额-本期账单余额'] = creditbill['上期还款金额'] - creditbill['本期账单余额']
creditbill['上期还款金额-信用卡额度'] = creditbill['上期还款金额'] - creditbill['信用卡额度']
creditbill['本期账单余额-信用卡额度'] = creditbill['本期账单余额'] - creditbill['信用卡额度']

creditbill.head()

In [ ]:
# use loop
cols = ['上期账单金额', '上期还款金额', '本期账单余额', '信用卡额度', '上期账单金额-上期还款金额', '上期账单金额-本期账单余额', '上期账单金额-信用卡额度', '上期还款金额-本期账单余额', '上期还款金额-信用卡额度', '本期账单余额-信用卡额度']
for ic, col in enumerate(cols):
    for st in ['mean', 'min', 'max', 'std', 'sum', 'skew']:
        if st == 'skew':
            if col not in ['信用卡额度', '上期账单金额-上期还款金额', '上期账单金额-信用卡额度']:
                continue
        print('-'*80)
        print(col, st)
        var = creditbill.groupby(['用户标识', '银行标识'])[col].agg({col: st}).reset_index()
        var = var.pivot('用户标识', '银行标识', col).reset_index()
        var.columns = ['用户标识'] + ['银行标识%d_%s_%s' % (i, col, st) for i in range(13)]
        display(var.head(3))
        var.to_hdf('./feats_v6/creditbill_amt%d_%s.h5' %(ic+1, st), key='data', mode='w')
        print('creditbill_amt%d_%s.h5 saved\n' %(ic+1, st))

In [ ]:
def time_diff_stat(ser):
    ts = ser.values // (3600 * 24)
    ts_diff = np.diff(ts)
    if ts_diff.shape[0] < 2:
        return 0, 0, 0, 0
    else:
        return ts_diff.min(), ts_diff.mean(), ts_diff.std()

creditbill.sort_values(by=['用户标识', '账单时间戳'], inplace=True)
credit_time_diff = creditbill.groupby(['用户标识'])['账单时间戳'].apply(time_diff_stat).reset_index()
for i, st in enumerate(['min', 'mean', 'std']):
    credit_time_diff['账单时间间隔_' + st] = credit_time_diff['账单时间戳'].apply(lambda x: x[i])
del credit_time_diff['账单时间戳']
    
display(credit_time_diff.head(3))
credit_time_diff.to_hdf('./feats_v6/credit_time_diff.h5', key='data', mode='w')